# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modeling the Data based on required queries. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

### The image below is a screenshot of what the denormalized data in <font color=red>**event_datafile_new.csv**</font> looks like after the code above is run:

<img src="image_event_datafile_new.jpg">

## Setting up the database connection

#### Creating a Cluster

In [22]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, you need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [23]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [24]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## We want to answer the following three questions:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Question 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Based on the question, the table needs to contain the columns
- artist
- song title
- song's length

Because of the search criteria for retrieving these we also need the two columns
- session_id
- item_in_session

These two columns are going to be our primary key. We select session_id as our partition key, 
so that entries belonging to the same session are stored on the same node. The column item_in_session 
will be a clustering column, so we are able to use it in the WHERE clause of our query.

### Create table songplay_by_session based on these considerations

In [25]:
query = "CREATE TABLE IF NOT EXISTS songplay_by_session"
query = query + "(session_id int, item_in_session int, artist text, song text, length decimal, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

## Get data from CSV and insert it into songplay_by_session

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_by_session (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## SELECT statement to answer Question 1

In [27]:
query = "SELECT artist, song, length FROM songplay_by_session where session_id = 338 AND item_in_session = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '|',row.song, '|',row.length)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


# Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Based on the question, the table needs to contain the columns
- artist
- song title
- user first name
- user last name

Because of the search criteria for retrieving these we also need the two columns
- user_id
- session_id

For sorting the items by item in session, we need the column
- item_in_session

So that sessions from the same user are stored on the same node, we choose (user_id, session_id) 
as our partition key, and item_in_session as a clustering column.

### Create table songplay_by_user_session based on these considerations

In [28]:
query = "CREATE TABLE IF NOT EXISTS songplay_by_user_session"
query = query + "(user_id int, session_id int, item_in_session int, first_name text, last_name text, artist text, song text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

## Get data from CSV and insert it into songplay_by_user_session

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_by_user_session (user_id, session_id, item_in_session, first_name, last_name, artist, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

## SELECT statement to answer Question 2

In [30]:
query = "SELECT artist, song, first_name, last_name FROM songplay_by_user_session where user_id = 10 AND session_id = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '|', row.song, '|', row.first_name, row.last_name)

Down To The Bone | Keep On Keepin' On | Sylvie Cruz
Three Drives | Greece 2000 | Sylvie Cruz
Sebastien Tellier | Kilometer | Sylvie Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


# Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Based on the question, the table needs to contain the columns
- user first name
- user last name

Because of the search criteria for retrieving these we also need the column
- song

Because we need to identify users that have listened to a given song, we combine the column song with
- user_id

to form our primary key. While the combination of song and user is not going to be unique in the data 
(users may listen to the same song multiple times) it is enough to answer question 3, because we are not 
interested in how often or in which particular session users have listened to the song.

### Create table songplay_by_song based on these considerations

In [31]:
query = "CREATE TABLE IF NOT EXISTS songplay_by_song"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Get data from CSV and insert it into songplay_by_song

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_by_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## SELECT statement to answer Question 3

In [33]:
query = "SELECT first_name, last_name FROM songplay_by_song where song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [34]:
drop_songplay_by_session = "DROP TABLE IF EXISTS songplay_by_session;"
drop_songplay_by_user_session = "DROP TABLE IF EXISTS songplay_by_user_session;"
drop_songplay_by_song = "DROP TABLE IF EXISTS songplay_by_song;"
try:
    session.execute(drop_songplay_by_session)
    session.execute(drop_songplay_by_user_session)
    session.execute(drop_songplay_by_song)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [35]:
session.shutdown()
cluster.shutdown()